In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

# base path
base_path = f'{ontology}-test_site/test_build'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(base_path.split('/')[0],base_path.split('/')[1],'.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values: dict_values(['area-value'])
values: dict_values(['heating_capacity-value', 'cooling_COP-value', 'heating_COP-value', 'cooling_capacity-value'])
values: dict_values(['tolerance-value', 'resolution-value', 'setpoint_deadband-value', 'active-value', 'stage_count-value'])
values: dict_values(['area-value', 'azimuth-value', 'tilt-value'])
values: dict_values(['longitude-value', 'latitude-value', 'noaastation-value', 'timezone-value'])
{'site_id': 's223-test_site', 'hvac_type': 'hp-rtu', 'variable_params': {'hvac-zone': {}, 'space': {'area': '<name>-area'}, 'hp-rtu': {'cooling_capacity': '<name>-cooling_capacity', 'heating_COP': '<name>-heating_COP', 'heating_capacity': '<name>-heating_capacity', 'cooling_COP': '<name>-cooling_COP'}, 'tstat': {'setpoint_deadband': '<name>-setpoint_deadband', 'active': '<name>-active', 'tolerance': '<name>-tolerance

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey(base_path)

Loading config from: s223-test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 4 empty cells across 1 columns in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area']
Expanded CSV shape: (3, 13)
New columns added: ['cooling_capacity', 'heating_COP', 'heating_capacity', 'cooling_COP']
Expanded CSV shape: (3, 14)
New columns added: ['setpoint_deadband', 'active', 'tolerance', 'resolution', 'stage_count']
Expanded CSV shape: (7, 10)
New columns added: ['azimuth', 'area', 'tilt']
Expanded CSV shape: (1, 9)
New columns added: ['longitude', 'latitude', 'noaastation', 'timezone']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")

# Run inference on model
inferred_graph = handler.infer(s.graph)

inferred_graph.serialize(f'{base_path}/reasoned.ttl', format = 'ttl')
# if not validation_result.valid:
#     print("Validation failed:")
#     print(validation_result.report_string)

<Graph identifier=6a4d4c9d-9d47-4ad7-bb29-f4a815b30cdb (<class 'rdflib.graph.Graph'>)>

In [8]:
# lots of new inferred information
(inferred_graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-test_site#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_1> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_1-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_1-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_1-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_1-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_2> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_2-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_2-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_2-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_2-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_3> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_3-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_3-cooling_capacity>,
        

# Testing get Metadata

In [9]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{base_path}/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

In [10]:
df = loader._get_dataframe()

In [11]:
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 

main_entity_col = 'name'

# Mapping columns to templates (which are also HPFS types)
entity_types = {}
attr_types = {}

# Type of entity and types of related attributes
entity_attr_types = {}
entity_entity_types = {}

# # May delete, not sure if I need this
# entity_class_relation = {}
value_templates,entity_templates = get_template_types(ontology='s223')

# mapping entity cols to related point cols in df
entity_attr_cols = {}

# mapping entity cols to related entity cols in df
entity_entity_cols = {}
row = df.iloc[0]
# get all the entity types
for col, entity in row.items():
    for p, o in loader.g.predicate_objects(entity):
        # o is entity class
        if p == A and (loader.g.compute_qname(o)[1]) == URIRef(HPFS):
            entity_type = get_uri_name(loader.g,o)
            if entity_type in entity_templates:
                # getting entity types
                entity_types[col] = entity_type
            if entity_type in value_templates:
                # getting value types
                attr_types[col] = entity_type
            continue
        # o is an attribute of entity
        if p == HPFS['has-point']:
            # getting entity attr cols
            if col not in entity_attr_cols.keys():
                entity_attr_cols[col] = set()
            # Should always be a single value, may have to validate this
            col_names = row.index[row == o]
            if len(col_names) != 1:
                print(f'incorrect amount of columns returned for points {o}')
                print(col_names)
                print(o)
            else:
                entity_attr_cols[col].add(col_names.values[0])
        for p2, o2 in loader.g.predicate_objects(o):
            # getting entity relations if o is another defined entity
            if p2 == A and (loader.g.compute_qname(o2)[1]) == URIRef(HPFS):
                if o2 in [ HPFS[val] for val in entity_templates ]:
                    if col not in entity_entity_cols:
                        entity_entity_cols[col] = set()
                    # no self relations allowed
                    col_names = row.index[row == o]
                    if len(col_names) != 1:
                        print(f'incorrect amount of columns returned for {o}')
                    else:
                        col_value = col_names.values[0]
                        if col_value != col:
                            entity_entity_cols[col].add(col_value)

#change sets to iterables
entity_attr_cols = {entity:list(attrs) for entity, attrs in entity_attr_cols.items()}
entity_entity_cols = {entity:list(other_entities) for entity, other_entities in entity_entity_cols.items()}

# reshape direction relationships to make direction from main template of focus
reverse_related_to = []
for entity, entities in entity_entity_cols.items():
    if main_entity_col in entities:
        reverse_related_to.append(entity)
        entities.remove(main_entity_col)
if len(reverse_related_to) > 0:
    entity_entity_cols[main_entity_col] += reverse_related_to

# creating type dictionaries to dynamically instantiate classes
entity_attr_types = { 
    entity_types[entity_col]: [
        attr_types[val_col] for val_col in val_cols
    ] 
    for entity_col, val_cols in entity_attr_cols.items()
}
entity_entity_types = { 
    entity_types[entity_col]: [
        entity_types[other_entity_col] for other_entity_col in other_entity_cols
    ] 
    for entity_col, other_entity_cols in entity_entity_cols.items()
}


In [12]:
row.index

Index(['hvac', 'hvac_cooling_COP', 'hvac_cooling_COP_aspects_in',
       'hvac_cooling_COP_unit', 'hvac_cooling_COP_value',
       'hvac_cooling_capacity', 'hvac_cooling_capacity_aspects_in',
       'hvac_cooling_capacity_unit', 'hvac_cooling_capacity_value',
       'hvac_heating_COP', 'hvac_heating_COP_aspects_in',
       'hvac_heating_COP_unit', 'hvac_heating_COP_value',
       'hvac_heating_capacity', 'hvac_heating_capacity_aspects_in',
       'hvac_heating_capacity_unit', 'hvac_heating_capacity_value', 'name',
       'space', 'space_area', 'space_area_unit', 'space_area_value', 'tstat',
       'tstat_active', 'tstat_active_aspects_in', 'tstat_active_value',
       'tstat_resolution', 'tstat_resolution_unit', 'tstat_resolution_value',
       'tstat_setpoint_deadband', 'tstat_setpoint_deadband_aspects_in',
       'tstat_setpoint_deadband_unit', 'tstat_setpoint_deadband_value',
       'tstat_stage_count', 'tstat_stage_count_aspects_in',
       'tstat_stage_count_value', 'tstat_toleran

In [13]:
display(entity_entity_cols)
display(entity_attr_cols)
display(entity_entity_types)
display(entity_attr_types)


{'hvac': [], 'name': ['window', 'space', 'hvac', 'tstat'], 'tstat': []}

{'hvac': ['hvac_cooling_COP',
  'hvac_cooling_capacity',
  'hvac_heating_capacity',
  'hvac_heating_COP'],
 'space': ['space_area'],
 'tstat': ['tstat_setpoint_deadband',
  'tstat_resolution',
  'tstat_tolerance',
  'tstat_active',
  'tstat_stage_count'],
 'window': ['window_azimuth', 'window_tilt', 'window_area']}

{'hp-rtu': [],
 'hvac-zone': ['window', 'space', 'hp-rtu', 'tstat'],
 'tstat': []}

{'hp-rtu': ['cooling-COP',
  'cooling-capacity',
  'heating-capacity',
  'heating-COP'],
 'space': ['area'],
 'tstat': ['tstat-deadband',
  'tstat-resolution',
  'tstat-tolerance',
  'tstat-active',
  'tstat-stage_count'],
 'window': ['azimuth', 'tilt', 'area']}

In [14]:
print(site_info)

{'sites': [site(name='urn:hpflex/s223-test_site#s223-test_site', latitude=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), noaastation=Value(value=1.0, unit='None'), timezone=Value(value=1.0, unit='None'), longitude=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), )], 'zones': [hvac_zone(name='urn:hpflex/s223-test_site#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-test_site#zone_2', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-test_site#zone_1', , windows=2, spaces=2, hp_rtus=1, tstats=1)]}


In [15]:
zone = site_info['zones'][0]
print(zone)

hvac_zone(name='urn:hpflex/s223-test_site#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1)


In [16]:
zone.spaces

[space(name='urn:hpflex/s223-test_site#space_3_1', area=Value(value=4.0, unit='http://qudt.org/vocab/unit/FT2'), )]

In [17]:
[ zone.windows for zone in site_info['zones'] ]# [0]# .area.name

[[window(name='urn:hpflex/s223-test_site#window_3_3', azimuth=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'), tilt=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'), area=Value(value=7.0, unit='http://qudt.org/vocab/unit/FT2'), )],
 [window(name='urn:hpflex/s223-test_site#window_2_2', azimuth=Value(value=4.0, unit='http://qudt.org/vocab/unit/Degree'), tilt=Value(value=4.0, unit='http://qudt.org/vocab/unit/Degree'), area=Value(value=4.0, unit='http://qudt.org/vocab/unit/FT2'), )],
 [window(name='urn:hpflex/s223-test_site#window_1_2', azimuth=Value(value=2.0, unit='http://qudt.org/vocab/unit/Degree'), tilt=Value(value=2.0, unit='http://qudt.org/vocab/unit/Degree'), area=Value(value=2.0, unit='http://qudt.org/vocab/unit/FT2'), ),
  window(name='urn:hpflex/s223-test_site#window_1_1', azimuth=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), tilt=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), area=Value(value=1.0, unit='http://qudt.org/vocab/unit/

In [18]:
zone.tstats[0]

tstat(name='urn:hpflex/s223-test_site#tstat_zone_3', tstat_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_resolution=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_active=Value(value=3.0, unit='http://qudt.org/vocab/unit/NUM'), tstat_stage_count=Value(value=3.0, unit='http://qudt.org/vocab/unit/NUM'), )

In [19]:
print(zone.tstats[0].tstat_resolution)
zone.tstats[0].tstat_resolution.convert_to_si()
print(zone.tstats[0].tstat_resolution)
print(zone.tstats[0].tstat_resolution.is_delta)

Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F')
Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')
True


In [20]:
# optionally just load everything as si 
si_loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", ontology = 's223', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].tstat_resolution)

Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')


In [21]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

In [22]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

# Testing Unit Conversion

In [23]:
convert_units(10, 'FT', 'M')

3.048

In [24]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [25]:
convert_units(0, 'DEG_C', 'K')

273.15

In [26]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [27]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [28]:
# bg.create_dashboard('AQL from brick')

In [29]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')